# Training and Evaluating models:

## Pre-processing of data

In [3]:
import os
from torchvision.transforms import v2
import torchvision.transforms.functional as F
from PIL import Image

IMG_SIZE = 224
HALF = IMG_SIZE // 2

# Create transformation for images
pre_transform = v2.Compose([
    v2.Lambda(lambda img: F.affine(img, angle=0, translate=(-35, 0), scale=1, shear=0)), # Shift 35 pixels left (Correct center alignment)
    v2.Resize(IMG_SIZE, antialias=True), #Resize Image
    v2.CenterCrop(IMG_SIZE), #Crop Image
])


# Paths
src_root = '../../dataset'
dst_root = '../../dataset2'

# Create dataset2 directory structure
os.makedirs(dst_root, exist_ok=True)

# Loop through each class folder
for folder in os.listdir(src_root):
    folder_src = os.path.join(src_root, folder)
    folder_dst = os.path.join(dst_root, folder)
    os.makedirs(folder_dst, exist_ok=True)

    for filename in os.listdir(folder_src):
        if filename.lower().endswith('.jpg'):
            img_path = os.path.join(folder_src, filename)
            image = Image.open(img_path).convert('RGB')
            image = pre_transform(image)

            base_name = os.path.splitext(filename)[0]

            # TOP
            image_new = image.copy()
            image_part = image_new.crop((0, 0, IMG_SIZE, HALF))
            image_part.save(os.path.join(folder_dst, f"{base_name}_TOP.jpg"))

            # BOTTOM
            image_new = image.copy().rotate(180)
            image_part = image_new.crop((0, 0, IMG_SIZE, HALF))
            image_part.save(os.path.join(folder_dst, f"{base_name}_BOTTOM.jpg"))

            # LEFT
            image_new = image.copy().rotate(-90)
            image_part = image_new.crop((0, 0, IMG_SIZE, HALF))
            image_part.save(os.path.join(folder_dst, f"{base_name}_LEFT.jpg"))

            # RIGHT
            image_new = image.copy().rotate(90)
            image_part = image_new.crop((0, 0, IMG_SIZE, HALF))
            image_part.save(os.path.join(folder_dst, f"{base_name}_RIGHT.jpg"))


In [4]:
import pandas as pd

# Define Excel file path
excel_file = "../../Types_Cutting_Inserts.xlsx"
csv_file = "Wear_data.csv"
insert_type_list = ["RM121263NE-BB", "RM090955NE-AB", "RM090955NE-AC", "RM121279NE-CV", "RM121279NE-DF", "RM121279NE-CU", "SNC-44-170", "SNC-44-60KH04"]

# Read all sheets into a dictionary of DataFrames
sheets = pd.read_excel(excel_file, sheet_name=None)

# Create an empty list to store reshaped data
reshaped_data = []

# Process each sheet
for insert_type, df in sheets.items():
    if insert_type in insert_type_list:
        df = df.iloc[:, :5]
        df.columns = ["Insert_Name", "TOP", "LEFT", "RIGHT", "BOTTOM"]

        # Reshape the data
        for _, row in df.iterrows():
            insert_name = row["Insert_Name"]
            reshaped_data.extend([
                {"Insert_Name": f"{insert_name}_TOP", "Wear": row["TOP"]},
                {"Insert_Name": f"{insert_name}_LEFT", "Wear": row["LEFT"]},
                {"Insert_Name": f"{insert_name}_RIGHT", "Wear": row["RIGHT"]},
                {"Insert_Name": f"{insert_name}_BOTTOM", "Wear": row["BOTTOM"]},
            ])

# Convert reshaped data into a DataFrame
final_df = pd.DataFrame(reshaped_data)

# Save to CSV
final_df.to_csv(csv_file, index=False)
print(f"CSV file saved as {csv_file}")


CSV file saved as Wear_data.csv


## Training

In [ ]:
! python train.py \
    --model_name resnet18 \
    --ckpts_path ./ckpts \
    --data_path ../../dataset2 \
    --csv_path Wear_data.csv \
    --num_folds 5 \
    --val_fold 0 \
    --batch_size 128 \
    --val_batch_size 128 \
    --num_workers 0 \
    --lr 0.1 \
    --momentum 0 \
    --weight_decay 0 \
    --num_epochs 200 \
    --wandb_entity WANDB_ENTITY \
    --wandb_project WANDB_PROJECT \
    --loss_weight_class 1000.0 \
    --loss_weight_reg 1.0 \
    --split 0.9

c69ojbol
wandb: 
wandb: 🚀 View run brisk-microwave-153 at: https://wandb.ai/ian-boyen-ku-leuven/Masterproef/runs/c69ojbol
wandb: Find logs at: wandb\run-20250510_174150-c69ojbol\logs


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ian-boyen (ian-boyen-ku-leuven) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.8
wandb: Run data is saved locally in c:\Users\ianbo\Documents\school\1E-ICT\1Ma\Masterproef\Code\neural_networks\neural_network_multiTask2\wandb\run-20250510_174150-c69ojbol
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run brisk-microwave-153
wandb:  View project at https://wandb.ai/ian-boyen-ku-leuven/Masterproef
wandb:  View run at https://wandb.ai/ian-boyen-ku-leuven/Masterproef/runs/c69ojbol

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  3%|▎         | 1/39 [00:01<00:41,  1.08s/it]

  5%|▌         | 2/39 [00:01<00:30,  1.22it/s]

  8%|▊         | 3/39 [00:02<00:26,  1.34it/s]

 10%|█         | 4/39 [00:02<00:23,  1.46it/s]

 13%|█▎        | 5/3

# Evaluation

In [12]:
! python test.py \
    --model_name resnet18 \
    --load_ckpt ./ckpts/multiTask2_lr=0.1;epoch=200;loss_weight=1000.pth \
    --data_path ../../dataset2 \
    --csv_path Wear_data.csv \
    --num_workers 0 \
    --val_batch_size 32 \
    --split 0.9

Overall Report:
--------------------------------------------------
Classification Accuracy: 99.86%
Regression Accuracy Range (within 20.0 percentage or 50µm tolerance): 79.82%
Binary Regression Accuracy: (Threshold = 200µm): 91.18%
False Positive Rate (Not worn-out but detected): 3.10%
False Negative Rate (Worn-out but not detected): 5.71%
Mean Squared Error (MSE): 217.7334
Mean Absolute Error (MAE): 0.9640
Total Targets: 8084

Detailed Report:
--------------------------------------------------
Classification Report:
                precision    recall  f1-score   support

RM090955NE-AB       1.00      1.00      1.00        26
RM090955NE-AC       1.00      1.00      1.00        51
RM121263NE-BB       1.00      1.00      1.00       127
RM121279NE-CU       1.00      1.00      1.00        72
RM121279NE-CV       1.00      0.99      1.00       123
RM121279NE-DF       0.99      1.00      1.00       108
   SNC-44-170       1.00      1.00      1.00        57
SNC-44-60KH04       1.00      1.00 


100%|██████████| 22/22 [00:04<00:00,  4.68it/s]
